In [1]:
import random
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import pymysql
from PIL import Image, ImageDraw
from io import BytesIO
import time
import threading



# Lisää tämä osio saadaksesi tietokantayhteyden tiedostosta
from database_connection import create_conn

image_widget = widgets.Image()
limit_input = widgets.IntText(value=300, description='Raja-arvo:')
status_label = widgets.Label()
current_draw_thread = None
img_copy = None


def get_distinct_sensor_ids():
    conn = create_conn()
    query = "SELECT DISTINCT sensor_id FROM SensorData;"

    with conn.cursor(pymysql.cursors.DictCursor) as cursor:
        cursor.execute(query)
        result = cursor.fetchall()
        sensor_ids = [row['sensor_id'] for row in result]

    conn.close()
    return sensor_ids

def create_color_legend(color_map):
    labels = []
    for sensor_id, color in color_map.items():
        label = widgets.Label(value=f"{sensor_id}: ", layout=widgets.Layout(width="auto"))
        color_box = widgets.Box(layout=widgets.Layout(border=f"3px solid rgb{color}", width="20px", height="10px"))
        legend_item = widgets.HBox([label, color_box])
        labels.append(legend_item)
    return widgets.VBox(labels)

def update_color_legend(df):
    color_map = {}
    for index, row in df.iterrows():
        sensor_id = int(row.sensor_id)
        if sensor_id not in color_map:
            color_map[sensor_id] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    color_legend.children = create_color_legend(color_map).children

#start_date_input = widgets.DatePicker(value=datetime.now() - timedelta(days=7), description='Alkupäivä:')
#end_date_input = widgets.DatePicker(value=datetime.now(), description='Loppupäivä:')
color_legend = widgets.VBox([])

import threading

img_copy = None

def draw_thread(df, delay=0.0005):
    global img_copy
    img = Image.open('./kauppa.jpg')
    width, height = img.size

    x_offset = 112
    y_offset = 27
    x_max = 1076
    y_max = 563
    x_scale = 1140 / 10406
    y_scale = 566 / 5220

    def scale_coords(x, y):
        xr = (x * x_scale) + x_offset
        yr = (y * y_scale) + y_offset
        zr = 0
        return xr, yr, zr

    color_map = {}

    for index, row in df.iterrows():
        sensor_id = int(row.sensor_id)
        if sensor_id not in color_map:
            color_map[sensor_id] = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

        d = ImageDraw.Draw(img)

        (x, y, z) = scale_coords(row.x, row.y)
        d.rectangle((x, y, x + 4, y + 4), fill=color_map[sensor_id])

        img_copy = img.copy()

        # Update global image_widget value
        img_bytes = BytesIO()
        img_copy.save(img_bytes, format='PNG')
        img_bytes = img_bytes.getvalue()
        image_widget.value = img_bytes

        time.sleep(delay)

def draw(df):
    global current_draw_thread

    # Get image size with this method
    img = Image.open('./kauppa.jpg')
    width, height = img.size

    # If there's a running thread, stop it before starting a new one
    if current_draw_thread and current_draw_thread.is_alive():
        current_draw_thread._stop()

    # Create a new thread to handle the drawing and animation
    current_draw_thread = threading.Thread(target=draw_thread, args=(df,))
    current_draw_thread.start()

    image_widget.format = 'png'
    image_widget.width = width
    image_widget.height = height

    # Show image_widget
    image_widget.layout.visibility = 'visible'


def get_sensor_data(days, start_time, end_time, limit, sensor_ids, start_date, end_date):
    conn = create_conn()
    days_str = ','.join(str(day) for day in days)
    sensor_ids_str = ','.join(str(sensor_id) for sensor_id in sensor_ids)
    start_timestamp = pd.to_datetime(start_date).strftime('%Y-%m-%d %H:%M:%S')
    end_timestamp = pd.to_datetime(end_date).strftime('%Y-%m-%d %H:%M:%S')
    #query = f"SELECT * FROM SensorData WHERE DATE_FORMAT(timestamp,'%w') IN ({days_str}) AND TIME(timestamp) BETWEEN '{start_time}' AND '{end_time}' AND sensor_id IN ({sensor_ids_str}) AND timestamp BETWEEN '{start_timestamp}' AND '{end_timestamp}' LIMIT {limit};"
    query = f"SELECT * FROM SensorData WHERE WEEKDAY(timestamp) IN ({days_str}) AND TIME(timestamp) BETWEEN '{start_time}' AND '{end_time}' AND sensor_id IN ({sensor_ids_str}) AND timestamp BETWEEN '{start_timestamp}' AND '{end_timestamp}' AND x > 1000 LIMIT {limit};"

    
    print("Kysely:", query)  # Tulosta kysely

    with conn.cursor() as cursor:
        cursor.execute(query)
        result = cursor.fetchall()

    conn.close()

    if result:
        columns = ["timestamp", "x", "y", "z", "sensor_id"]
        df = pd.DataFrame(result, columns=columns)
        return df
    else:
        return pd.DataFrame()


def on_submit_button_clicked(button):
    clear_output(wait=True)
    status_label.value = "Suoritetaan kyselyä..."
    display(ui)

    days = day_selector.value if day_selector.value else list(range(7))
    start_time = start_time_input.value
    end_time = end_time_input.value
    limit = limit_input.value
    sensor_ids = sensor_id_selector.value if sensor_id_selector.value else get_distinct_sensor_ids()
    alkupaiva_str = alkupaiva_input.value
    maksimipaiva_str = maksimipaiva_input.value
    start_date = pd.to_datetime(alkupaiva_str, format='%d.%m.%Y')
    end_date = pd.to_datetime(maksimipaiva_str, format='%d.%m.%Y')
    df = get_sensor_data(days, start_time, end_time, limit, sensor_ids, start_date, end_date)
    
    if not df.empty:
        update_color_legend(df)
        draw(df)
    else:
        status_label.value = "Ei tuloksia annetuilla parametreilla."

    print(df.head())

    status_label.value = ""



day_selector = widgets.SelectMultiple(
options=[('Maanantai', 1), ('Tiistai', 2), ('Keskiviikko', 3), ('Torstai', 4), ('Perjantai', 5), ('Lauantai', 6), ('Sunnuntai', 0)],
description='Viikonpäivät:',
)

alkupaiva_input = widgets.Text(value='13.3.2019', description='Alkupäivä:')
maksimipaiva_input = widgets.Text(value='23.3.2019', description='Loppupäivä:')

start_time_input = widgets.Text(value='11:01:00', description='Alkuaika:')
end_time_input = widgets.Text(value='17:56:59', description='Loppuaika:')

submit_button = widgets.Button(description='Piirrä pisteet')

submit_button.on_click(on_submit_button_clicked)

image_widget.layout.visibility = 'hidden'

color_legend = widgets.HTML()

sensor_id_options = [(str(sensor_id), sensor_id) for sensor_id in get_distinct_sensor_ids()]

sensor_id_selector = widgets.SelectMultiple(
    options=sensor_id_options,
    description='Sensorit:',
)


ui = widgets.VBox([
day_selector,
alkupaiva_input,
maksimipaiva_input,
start_time_input,
end_time_input,
limit_input,
sensor_id_selector,
submit_button,
status_label,
color_legend,
image_widget
])

display(ui)
